In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Get and pre-process the dataset

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

cols = ['PostalCode', 'Borough', 'Neighborhood']

# create a list, populate it firtst and then use it to create dataframe after the loop
lst = []

res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')

df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    
    # eliminate records with non assigned Borough values
    # the code also assignes Borough values to Neighborhood where it is not assigned
    if "Not assigned" not in data[2].text:     
        try:
            postal_code = data[0].text
            
            if "href=" in data[1].text:
                borough = data[1].a.text
            else:
                borough = data[1].text
                
            if "Not assigned" in data[2].text:   
                neigb = borough
            else:
                if "href=" in data[2].text:
                    neigb = data[2].a.text
                else:
                    neigb = data[2].text       

        except IndexError:pass
        lst.append([postal_code,borough,neigb])
        
df = pd.DataFrame(lst, columns=cols)
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
df.shape



(102, 3)

# Get a second dataset with postalcodes and merge 2 files

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
# the body parameter below is from the cell that hidden cell that has the credentials
df_data_1 = pd.read_csv(body)
df_data_1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
df_merged = pd.merge(df, df_data_1, left_on='PostalCode', right_on='Postal Code', how='left')

In [6]:
df_merged.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge\n,Malvern\n",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill\n",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn\n,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,M1H,43.773136,-79.239476
